In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import simps, quad
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import time
from datetime import datetime
import pytz

pi = np.pi

# Welcome to the Gross-Pitaevskii solver

This code is designed to solve the Gross-Pitaevskii equation for a Bose-Einstein condensate confined in a 3d isotropic harmonic potential. The numerical approach is based on the Crank-Nicolson method, incorporating an imaginary time evolution to obtain the ground state of the system.

Before running the simulation, you may need to define the relevant global parameters, like the number of particles of the system `N`, the s-wave scattering length `As`, the timestep and grid spacing, `dt`, `dr`, or the final point of the grid `r_max`. Since the density profile rapidly decays to zero at sufficiently large values of r, a `r_max` value of roughly 2-5 times the associated Thomas-Fermi lengthscale should be enough for most of the computations, though this may vary depending on the initial conditions.

The parameters are in harmonic oscillator units, such that $\hbar=m=\omega=1$.


In [ ]:
#global parameters
N = 1e4 #number of particles
As=0.00433 #s-wave scattering length. For As=0 we recover the known ground state solution of the quantum h.o
g = 4.*pi*As*N #interaction strength
dt = 1e-4 #timestep
dr=0.05 #grid spacing
#be careful for what values of dt and dr you choose, since some of them may yield numerical instabilities.
#As a check, you may need to see if the errorVirial parameter is sufficiently small when the error gets smaller than the tolerance parameter. If not, change the values of dt and dr
#to get the final results in a reasonable amount of time and with a sufficiently small errorVirial parameter.

#Thomas-Fermi parameters muTF and rTF
muTF = 0.5*(15*As*N)**(2./5.)
rTF = np.sqrt(2.*muTF)
print("Thomas-Fermi length is: ", rTF)

r_max = 2*rTF

Ngrid = int(r_max/dr)
r = np.linspace(0.,r_max,Ngrid+1)[1:] #we avoid r=0

Niter = 100000 #maximum number of timesteps allowed (safer)

The computation time depends highly on the number of grid points or the timesteps needed to achieve the tolerance wanted. It may take from a few seconds to several minutes or even a few hours, in the most extreme cases, to obtain the final results. If the time taken to do a computation is unreasonably large, you may:
- increase slightly the grid spacing `dr` (note that changing significantly this value may trigger numerical instabilities or higher than expected errors for the relevant quantities calculated)
- reduce slightly the value of the final point of the grid `r_max` (note that reducing significantly this value may involve higher errors in the calculation of the normalization integrals and thus the values of the relevant energies per particle, chemical potential etc.)


The ground state of the system is obtained through an iterative procedure based on the Crank-Nicolson implicit method, which solves for the radial part of the wave function $P(r)$ at each time step, where $\psi(r) = \frac{P(r)}{r}\frac{1}{\sqrt{4\pi}}$, and ending the loop when the "error" made, which is defined as $\varepsilon = \sum_i|P_i^\text{new} - P_i^\text{old}|$, where $(i\in\text{grid})$, gets smaller than the tolerance parameter `tol`. When the simulation ends, the vectors `r` and `P` will be written in a file called "r_P(As=...)(N=...)... .dat".

The last part of the code will be the plotting of the density profile compared with that obtained from the Thomas-Fermi approximation. For this, we may read the data of the file "r_P... .dat" and assign it to the vectors `r` and `P`, for later plotting and calculation of the desired energies per particle: total, kinetic, potential, interaction, and the chemical potential, as well as testing that the virial theorem is fulfilled within a certain tolerance. These two parts; the time evolution and the plotting, are separated since one might only want the plotting part without having to run the whole time evolution.

If you have any questions, suggestions, or issues regarding the solver, feel free to contact me at matteomarrone27@gmail.com.

In [ ]:
#TIME EVOLUTION PART OF THE CODE:

#We get the actual time of the start of the computation (change location from Europe/Madrid to whichever fits you best) 
tz = pytz.timezone('Europe/Madrid')
timenow = datetime.now(tz)
print("Current time in Barcelona: ", timenow.strftime("%H:%M:%S"))

#harmonic potential
V = 0.5*(r**2.)

#laplacian matrix in finite differences
D2 = (-2.*np.eye(Ngrid) + np.eye(Ngrid, k=1) + np.eye(Ngrid, k=-1))/(dr**2.)

#we construct the P radial function of the initial wave function, where psi = (P/r)/sqrt(4pi)
P = r*np.exp(-0.5*r**2.)
P  /= np.sqrt(simps(P**2, r)) #normalization

#imaginary time evolution
tol = 1e-6
error = 1.0
errorVirial = 1.
timesteps = 0

start_time = time.time()

while error>tol and timesteps<Niter:
    P_old = P.copy()

    psi = P/(r*np.sqrt(4.*pi)) #wave function
    G = g*np.abs(psi)**2. #non linear term

    #We construct the Cranc Nicolson matrices
    A = np.eye(Ngrid) + (dt/2.)*(-0.5*D2 +np.diag(V+G))
    B = np.eye(Ngrid) - (dt/2.)*(-0.5*D2 +np.diag(V+G))

    #solve the system
    P = np.linalg.solve(A, B @ P)
    P /= np.sqrt(simps(P**2, r)) #renormalize

    #calculate the density
    psi = P/(r*np.sqrt(4.*pi))
    dens = np.abs(psi)**2
    #we calculate the expected values of the energies
    Ekin = simps(-0.5*P*np.dot(D2,P), r)
    Epot = simps(V*P**2, r)
    Eint = simps(0.5*g*np.abs(psi)**4*(4.*pi*r**2), r)

    #compute error
    error = np.sum(np.abs(P-P_old))
    errorVirial = np.abs(-2*Epot+2*Ekin+3*Eint)
    timesteps+=1

    if timesteps % 400 == 0:
        print(f"step: {timesteps}, error: {error:.6e}, errorVirial: {errorVirial:.6e}")

print("Computation time (seconds) is: ", time.time()-start_time)
print("--------------------------------")

In [ ]:
#we write the computed data to the file
filename = f"r_P_(As={As})(N={N})(dt={dt})(dr={dr})(rmax={r_max})(tol={tol}).dat"
f = open(filename, 'w')

for i in range(len(r)):
    f.write(str(r[i])+" "+str(P[i]))
    f.write("\n")

f.close()

In [ ]:
#we read the computed data to plot it
with open(filename) as f:
    lines = f.readlines()
    r = np.array([line.split()[0] for line in lines],dtype=float)
    P = np.array([line.split()[1] for line in lines],dtype=float)

#calculate the density
psi = P/(r*np.sqrt(4.*pi))
dens = np.abs(psi)**2

print("Is the numerical solution normalized to 1?: ", simps(4.*pi*r**2 * np.abs(psi)**2, r))

#we calculate the expected values of the energies
Ekin = simps(-0.5*P*np.dot(D2,P), r)
Epot = simps(V*P**2, r)
Eint = simps(0.5*g*np.abs(psi)**4*(4.*pi*r**2), r)
mu = Ekin+Epot+2*Eint
Etotal = Ekin+Epot+Eint

print("Ekin: ", Ekin, "Epot: ", Epot, "Eint: ", Eint, "Etotal: ", Etotal)
print("virial?: ", -2*Epot+2*Ekin+3*Eint)
print("chem potential: ", mu)

#We determine the Thomas Fermi density function
TFdens = np.where(r<rTF, (muTF-0.5*r**2.)/g, 0.) 
TFdens /= simps(4.*pi*TFdens*r**2,r)
print("tf normalization: ", simps((4.*pi*r**2)*TFdens,r))

# Plotting
plt.plot(r,TFdens, label='Thomas-Fermi',color='red')
plt.plot(r, dens, label="Numerical solution")
plt.xlabel("r")
plt.ylabel(r"$|\psi(r)|^2$")
plt.title(f"Ground State of Gross-Pitaevskii Equation for N={N}")
plt.grid(True)
plt.legend()
plt.show()

#we write the important data involving energy values and virial error on another file
filename2 = f"final_res(As={As})(N={N})(dt={dt})(dr={dr})(rmax={r_max})(tol={tol}).dat"
f2 = open(filename2,'w')

f2.write("Ekin= "+str(Ekin)+" "+"Epot= "+str(Epot)+" "+"Eint= "+str(Eint))
f2.write("\n")
f2.write("mu= "+str(mu)+" "+"Etotal= "+str(Etotal)+" "+"virialErr= "+str(-2*Epot+2*Ekin+3*Eint))
f2.close()